对于每一个投资者<br/>
T+1日看涨/看跌行为 对 T-1日到T日结算价变化方向 做softmax回归

| 结算价变化 | 变量 |
| ---------- | ---- |
|   上涨   |  +1  |
|   持平   |  0  |
|   下跌   |  -1  |

|持仓量变化|行为类型|变量|
|----------|--------|----|
|买持仓增加|看涨|1|
|买持仓减少|看跌|-1|
|卖持仓增加|看跌|-1|
|卖持仓减少|看涨|1|
|持仓，但持仓不变 |观望|0|

In [1]:
import sys
sys.path.append('../my/')
from db import pool
import pandas as pd
from sklearn.linear_model import LogisticRegression as Logit

In [ ]:
softmax = Logit(multi_class='multinomial', solver='sag', n_jobs=-1)

rise = (1., 1.)
fall = (-1., 1.)
same = (0., 1.)

conn = pool.dedicated_connection()
cursor = conn.cursor()

cursor.execute("use investor")

# 账号
accounts = []
sql = 'select distinct account from chicang'
cursor.execute(sql)
accounts = [row[0] for row in cursor.fetchall()]

# 日期 (昨天、今天、明天)
days = []
lastdays = []
nextdays = []
sql = "select day,lastday,nextday from commodity.trade_day where day between '20140101' and '20161231' order by day asc"
cursor.execute(sql)
for row in cursor.fetchall()[1:-1]:
    days.append(row[0])
    lastdays.append(row[1])
    nextdays.append(row[2])

for account in accounts:
    X = []
    Y = []
    for i in range(len(days)):
        day = days[i]
        lastday = lastdays[i]
        nextday = nextdays[i]
        #  今日持仓合约 或 明日持仓合约
        sql = "(select distinct vari_code,deliv_date from chicang where account=%s and tradedate=%s) union \
        (select distinct vari_code,deliv_date from chicang where account=%s and tradedate=%s)"
        cursor.execute(sql, (account,day,account,nextday))
        contracts = cursor.fetchall()
        for contract in contracts:
            # 结算价（要求昨天和今天都有结算价）
            sql = "select settle from commodity.contract_daily where day=%s and code=%s and deli=%s"
            if cursor.execute(sql, (day,contract[0],contract[1]))==0:
                continue
            settle = cursor.fetchone()[0]
            if cursor.execute(sql, (lastday,contract[0],contract[1]))==0:
                continue
            settle_last = cursor.fetchone()[0]
            # 确定回归元
            if settle>settle_last:
                x = 1.
            elif settle==settle_last:
                x = 0.
            else:
                x = -1.
            x = (x, 1.)    # 截距项
            # 持仓
            sql = "select sum(qty) from chicang where account=%s and tradedate=%s and vari_code=%s and deliv_date=%s and bs_flag=%s"
            for bs in ('0','1'):
                # 今日持仓量
                cursor.execute(sql, (account, day, contract[0], contract[1], bs))
                oi = cursor.fetchone()[0]
                if oi is None:
                    oi = 0
                # 明日持仓量
                cursor.execute(sql, (account,nextday,contract[0],contract[1],bs))
                oi_next = cursor.fetchone()[0]
                if oi_next is None:
                    oi_next = 0
                # 排除两日没有持仓的情况
                if oi==0 and oi_next==0:
                    continue
                # 行为分析
                if bs=='0' and oi_next>oi:
                    y = 1
                elif bs=='0' and oi_next<oi:
                    y = -1
                elif bs=='1' and oi_next>oi:
                    y = -1
                elif bs=='1' and oi_next<oi:
                    y = 1
                else:
                    y = 0
                X.append(x)
                Y.append(y)
    samples = len(X)
    if samples<20:
        continue
    # softmax
    softmax.fit(X, Y)
    result = pd.DataFrame(softmax.predict_proba([rise, fall, same]), columns=softmax.classes_, index=['raise','fall','same'])
    if 1 not in result.columns:
        result[1] = 0.
    if 0 not in result.columns:
        result[0] = 0.
    if -1 not in result.columns:
        result[-1] = 0.
    # 导入数据库
    sql = "insert into settle_change_direction2behavior_softmax values (" + "%s,"*7 + "%s)"
    cursor.execute(sql, [account] + list(map(lambda x:float(x), [result.loc['raise',1], result.loc['raise',-1], result.loc['same',1], 
                                                            result.loc['same',-1], result.loc['fall',1], result.loc['fall',-1]])) + 
                                        [int(samples)])
    conn.commit()

cursor.close()
conn.close()
print ('finish!')